# Thermal background of Fiber IRMOS

Nick Konidaris
6 June 2018

In [202]:
import numpy as np
%matplotlib inline
from pylab import *

import scipy.integrate
import scipy




from collections import namedtuple

import numpy as np
%matplotlib inline
from pylab import *

import astropy
from astropy import units as ur
from astropy import constants as cc
import astropy.modeling.blackbody



## Introduction

The thermal background of fiber irmos is computed.

Here I tabulate the Étendue of various systems preceeindg the fiber tip. These Etendues are useful because one can add them together and multiply by the blackbody spectral radiance to compute a thermal background for the instrument. Recall that for some spectral radiance $L_{e \Omega \lambda}$ in units of $\left[ Power/Bandwidth/Area/Solid\ angle \right]$ there is an Étendue ($G$) that will yield a Spectral flux $\Phi_e = L_{e \Omega \lambda} \cdot G$. 

Our goal with fiber IRMOS is to produce an instrument whose Étendue from the spectrograph is only a fraction of the Étendue from the telescope. So it is worth computing the Étendue of the telescope first. I use various approximations that are useful.

$\Omega \approx \pi \sin^2 \theta$ and this is equivalent to $\Omega \approx \frac{\pi}{(2\cdot f/\#)^2}$

$G_{telescope} = \Omega_{telescope} \cdot A_{telescope} \cdot \epsilon_{telescope} = 3.61E-6\cdot cm2\cdot sr$ (see below)


In [203]:
def theta_to_omega(theta_rad):
    
    return np.pi * np.sin(theta_rad)**2 * ur.steradian

def fnum_to_omega(fnum):
    return np.pi /(2*fnum)**2 * ur.steradian

In [204]:
# Important constants


ϵ_world = 1

# FOREOPTICS
final_focal_ratio = 3.5 # This is the final focal ratio entering the fiber
fiber_focal_ratio = 3.0 # Given by the manufacturer
refl_ar_coating = 0.99 # I hope


# FIBER
fiber_ffov = 0.8 * ur.arcsecond
fiber_diameter = 100 * ur.micron

# TELESCOPE
Dtel = 6.5 * ur.meter
n_mirrors = 3
refl_mirror = .9
ϵ_secondary = 0.1 ## WAG
fnum_tel = 11

#### DERIVED
Atel = (np.pi * (Dtel/2)**2).to(ur.cm**2)*.93
Asecondary = Atel * 0.074 # from http://www.lco.cl/telescopes-information/magellan/magellan-telescopes/#figure1
ϵ_tel = 1 - refl_mirror**n_mirrors
A_fiber = np.pi*(fiber_diameter/2)**2
Gus = ur.cm**2 * ur.sr

def p_h(varname):
    val = eval(varname)
    print("{0:20s} is {1:1.2e} or {2:3.0f}% of telescope".format(varname, val, val/G_telescope*100))

In [205]:
G_telescope = (theta_to_omega(fiber_ffov) * Atel * ϵ_tel).to(Gus)

p_h("G_telescope")

G_telescope          is 3.95e-06 cm2 sr or 100% of telescope


###  Fiber etendue -- first order

For the purposes of thermal background, I am most concerned with the étendue of the area outside the controlled f/# as defined by the manufacturer of the fiber. I also compute the étendue of the secondary assuming the spectrograph's internal Lyot stop will not block the secondary. I assume the world has unit emissivity.


In [206]:
G_secondary = (fnum_to_omega(fnum_tel) * A_fiber * ϵ_secondary).to(Gus)
G_fiber = (np.abs(fnum_to_omega(final_focal_ratio) - fnum_to_omega(fiber_focal_ratio)) * A_fiber * ϵ_world).to(Gus)

p_h("G_secondary")
p_h("G_fiber")

G_secondary          is 5.10e-08 cm2 sr or   1% of telescope
G_fiber              is 1.82e-06 cm2 sr or  46% of telescope


###  Fiber etendue -- second order

0th order attributes of a fiber sytem are shown in the figure below under the assumption that a ball lens is used to reduce the focal ratio of the telescope. 

![0th order fiber trace](fiber_0th_order.png)

While the 0th order system is a useful place to start, there are ghost images that appear after 1 bounce. These are "first order ghosts" and are shown in the image below

![first order ghost fiber trace](fiber_1st_order.png)

If I raytrace $10^6$ rays and give the ball lens a 50% coating then zemax reports $147,854$ rays return in a 1st order ghost. With a 99% coating zemax reports $9,720$ rays are sent to the 1st order ghost. 

Note that only odd-order ghosts go to the "left" in the figure above, and even-order ghosts go to the right. As the ghost order increases the rays bounce around more and become more angularly homogeneous, so to overestimate their contribution I assume any ghosts higher than second order come from $4\pi$. It turns out that even with their solid angle overestimated, ghosts higher than 1st order benefit from $\epsilon^{ghost\ order}$ and so only the first-order ghosts matter. 

In [217]:
G_first_order = (theta_to_omega(36*ur.degree) * A_fiber * (1-refl_ar_coating)).to(Gus)
G_second_order = (theta_to_omega(36*ur.degree) * A_fiber * (1-refl_ar_coating)**2).to(Gus)
G_third_order = (4*np.pi*ur.sr * A_fiber * (1-refl_ar_coating)**3).to(Gus)

p_h("G_first_order")
p_h("G_second_order")
p_h("G_third_order")

G_first_order        is 8.52e-07 cm2 sr or  22% of telescope
G_second_order       is 8.52e-09 cm2 sr or   0% of telescope
G_third_order        is 9.87e-10 cm2 sr or   0% of telescope


In [218]:


vs = ["G_telescope", "G_secondary", "G_fiber", "G_first_order", "G_second_order", "G_third_order"]
for v in vs:
    p_h(v)

G_telescope          is 3.95e-06 cm2 sr or 100% of telescope
G_secondary          is 5.10e-08 cm2 sr or   1% of telescope
G_fiber              is 1.82e-06 cm2 sr or  46% of telescope
G_first_order        is 8.52e-07 cm2 sr or  22% of telescope
G_second_order       is 8.52e-09 cm2 sr or   0% of telescope
G_third_order        is 9.87e-10 cm2 sr or   0% of telescope
